In [1]:
#reddit account
#username:IES_Python
#email:95346725@fsv.cuni.cz

In [1]:
#pip install praw
import praw
import time
from praw.models import MoreComments
import pandas as pd
from datetime import date
credentials = {
    "client_id": "Mz-eGRSHPkpkaQ",
    "client_secret": "hNHgiDETfcYvbUweNZrSV-dAHt_mbw",
    "user_agent": "IES_python_stocks",
    "username": "IES_Python",
    "password": "IES_python"
}

In [2]:


class Reddit_Extractor:
    def __init__(self,credentials):
        self.reddit = self.redditApi()
        
    def redditApi(self):
        reddit = praw.Reddit(
            client_id = credentials['client_id'],
            client_secret = credentials['client_secret'],
            user_agent = credentials['user_agent'],
            username = credentials['username'],
            password = credentials['password'])
        return reddit
    
    def subredditData(self,subreddit,limit = 10):
        postList = self.reddit.subreddit(subreddit).hot(limit=limit)
        return postList     
        
    def showcasePrint(self,postList):
        for post in postList:
            print(post.title)
            print(post.distinguished)
            print(post.link_flair_text)
            '''for top_level_comment in submission.comments:
                print(i)
                print(top_level_comment.link_id)   
                print(top_level_comment.body)      
                print(top_level_comment.created_utc)
                i+=1'''    
            
    def subredditDF(self,postList):
        df = pd.DataFrame({'post_id': pd.Series([], dtype='str'),
                        'comment_id': pd.Series([], dtype='str'),
                        'subreddit': pd.Series([], dtype='str'),
                        'text_type': pd.Series([], dtype='str'),
                        'epoch_time': pd.Series([], dtype='int'),
                        'text': pd.Series([], dtype='str'),
                        'post_title': pd.Series([], dtype='str')})
        for post in postList:
            #post extraction
            post_row = pd.DataFrame({'post_id': pd.Series([post.id], dtype='str'),
                                    'comment_id': pd.Series([], dtype='str'),
                                    'subreddit': pd.Series([post.subreddit.display_name], dtype='str'),
                                    'text_type': pd.Series(['post'], dtype='str'),
                                    'epoch_time': pd.Series([post.created_utc], dtype='int'),
                                    'text': pd.Series([post.selftext], dtype='str'),
                                    'post_title': pd.Series([post.title], dtype='str')})   
            df = df.append(post_row) 
            #comments extraction
            submission = self.reddit.submission(id=post.id)
            submission.comment_sort = 'best'
            submission.comment_limit = 10
            for top_level_comment in submission.comments:
                if isinstance(top_level_comment, praw.models.MoreComments):
                    continue                    
                new_row = pd.DataFrame({'post_id': pd.Series([post.id], dtype='str'),
                                        'comment_id': pd.Series([top_level_comment.id], dtype='str'),
                                        'subreddit': pd.Series([top_level_comment.subreddit.display_name], dtype='str'),
                                        'text_type': pd.Series(['comment'], dtype='str'),
                                        'epoch_time': pd.Series([top_level_comment.created_utc], dtype='int'),
                                        'text': pd.Series([top_level_comment.body], dtype='str'),
                                        'post_title': pd.Series([], dtype='str')})
                df = df.append(new_row)  
                                         
        return df
    def joinData(self,subreddits,limit):
        main_df = pd.DataFrame({'post_id': pd.Series([], dtype='str'),
                'comment_id': pd.Series([], dtype='str'),
                'subreddit': pd.Series([], dtype='str'),
                'text_type': pd.Series([], dtype='str'),
                'epoch_time': pd.Series([], dtype='int'),
                'text': pd.Series([], dtype='str'),
                'post_title': pd.Series([], dtype='str')})
        for subreddit in subreddits:
            posts_test = self.subredditData(subreddit = subreddit,limit = limit)
            start = time.time()
            df = self.subredditDF(posts_test)
            end = time.time()
            print("Elapsed time for subreddit",subreddit)
            print(end-start)
            main_df = main_df.append(df)
        return main_df

class TickerExtractor:
    def findNewTicker(self, post): #method to expand possible dictionaries of stock tickers
        pass
    
    def tickerCounter(self, tickerDF): #append number of mentions to dataframe
        #various aggregations of main df
        pass



In [3]:
#load posts
subreddits = ["stocks","investing","StockMarket","wallstreetbets"]
extr = Reddit_Extractor(credentials)
main_df = extr.joinData(subreddits,100)
#posts_test = extr.subredditData(subreddit = 'stocks',limit = 10)
#post_extr.showcasePrint(posts_test)

Elapsed time for subreddit stocks
35.240411043167114
Elapsed time for subreddit investing
35.509501934051514
Elapsed time for subreddit StockMarket
39.725247859954834
Elapsed time for subreddit wallstreetbets
100.68006181716919


In [4]:
#observe dataframe
#main_df.shape
main_df['subreddit'].value_counts()
#main_df.columns
#main_df.loc[main_df['text_type'] == 'comment']
#main_df.iloc[0]['text']

wallstreetbets    686
investing         634
stocks            592
StockMarket       508
Name: subreddit, dtype: int64

In [5]:
#store object
today = date.today()
stringEpoch = today.strftime("%d_%m_%y")
s = ''
nameVars = ["D:\\School\\bartusek_cala_python\\daily_reddit_data\\",stringEpoch,".csv"]
main_df.to_csv(path_or_buf=s.join(nameVars),index=False,encoding='utf-8-sig')

In [8]:
main_df.loc[main_df['post_id'] == 'm4rz5b']

,post_id,comment_id,subreddit,text_type,epoch_time,text,post_title
0,m4rz5b,NaN,stocks,post,1615713609,"The familiar [""Rate My Portfolio"" sticky can b...",r/Stocks Daily Thread on Meme Stocks Sunday - ...
0,m4rz5b,gqwajyw,stocks,comment,1615729382,Why the hell is there a daily meme stock threa...,NaN
0,m4rz5b,gqwezb9,stocks,comment,1615732573,Got my deposit and itching to play this weeken...,NaN
0,m4rz5b,gqx4lyb,stocks,comment,1615744711,I kind of feel like I might be better off inve...,NaN
0,m4rz5b,gqwbi0r,stocks,comment,1615730174,What do you guys think of lumber and steel sto...,NaN
0,m4rz5b,gqx84qi,stocks,comment,1615745939,XOM going to be a good buy for the year. Oil ...,NaN
0,m4rz5b,gqvxz7m,stocks,comment,1615716231,So if GME becomes a fulfillment center we’re n...,NaN
0,m4rz5b,gqwbqc9,stocks,comment,1615730345,"Dfv sold his options. 4/16 $12 oi is 530, of w...",NaN
